# Pre-Processing

In [ ]:
# If you don’t already have pandas installed, uncomment:
# pip install pandas

import os
import random
import pandas as pd

# ─── Config ────────────────────────────────────────────────────────────────────
NUM_POS       = 8827
NUM_NEG       = 1050302
TRAIN_FRAC_POS = 0.8    # 80% of positives → train
SEED          = 42      # reproducibility
PAD_WIDTH     = 5       # zero‑pad numeric IDs to at least 5 digits

# ─── Output paths ──────────────────────────────────────────────────────────────
base_dir  = "/Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data"
csv_train = os.path.join(base_dir, "TestTrainSplits", "train_test_easy", "train-3000.csv")
csv_test  = os.path.join(base_dir, "TestTrainSplits", "train_test_easy", "test-3000.csv")

random.seed(SEED)

# ─── 1) Build zero‑padded name lists ───────────────────────────────────────────
positives = [f"P{i:0{PAD_WIDTH}d}" for i in range(1, NUM_POS + 1)]
negatives = [f"N{i:0{7}d}" for i in range(1, NUM_NEG + 1)]

# ─── 2) 80/20 split on positives ───────────────────────────────────────────────
num_train_pos = int(len(positives) * TRAIN_FRAC_POS)
train_pos     = set(random.sample(positives, num_train_pos))
test_pos      = set(positives) - train_pos

# ─── 3) Sample negatives to get 50:50 balance ─────────────────────────────────
train_neg      = set(random.sample(negatives, len(train_pos)))
remaining_negs = set(negatives) - train_neg
test_neg       = set(random.sample(remaining_negs, len(test_pos)))

# ─── 4) Build, shuffle, and save splits ───────────────────────────────────────
def save_split(pos_set, neg_set, out_path):
    df = pd.DataFrame({
        "name":       list(pos_set) + list(neg_set),
        "is_positive": [True]*len(pos_set) + [False]*len(neg_set)
    })
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    df.to_csv(out_path, index=False)
    pct = df["is_positive"].mean() * 100
    print(f"→ {os.path.basename(out_path)}: {len(df)} rows, {pct:.1f}% positive")

save_split(train_pos, train_neg, csv_train)
save_split(test_pos,  test_neg,  csv_test)


→ train-3000.csv: 14122 rows, 50.0% positive
→ test-3000.csv: 3532 rows, 50.0% positive


/var/folders/f8/2jz1j73d6kl_wvvcg3ld5d_00000gn/T/ipykernel_1769/2351497194.py:34: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_neg       = set(random.sample(remaining_negs, len(test_pos)))


In [ ]:
import os
from datasets import get_dataloader, custom_collate_fn

# =============================================================================
# Adjust these paths according to your folder structure.
# =============================================================================
if __name__ == '__main__':
    # For the train_test_easy split
    base_dir = "/Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data"
    
    # Use the CSV files in the easy split folder:
    csv_train = os.path.join(base_dir, "TestTrainSplits", "train_test_easy", "train-3000.csv")
    csv_test  = os.path.join(base_dir, "TestTrainSplits", "train_test_easy", "test-3000.csv")
    
    # Directory containing JPEG images.
    images_dir = os.path.join(base_dir, "JPEGImageFull", "dataset", "JPEGImage")
    # Directory containing positive XML annotations.
    annotations_dir = os.path.join(base_dir, "positive-Annotation")
    
    # DataLoaders for training and testing.
    # Pass the custom collate function here:
    train_loader = get_dataloader(csv_train, images_dir, annotations_dir, batch_size=32, train=True)
    test_loader  = get_dataloader(csv_test, images_dir, annotations_dir, batch_size=32, train=False)
    
    # When creating the DataLoader inside get_dataloader, set the collate_fn parameter
    
    # For our testing, we can either modify get_dataloader() or wrap it here:
    from torch.utils.data import DataLoader
    # Reconstruct using our custom_collate_fn for demonstration:
    train_loader = DataLoader(train_loader.dataset, batch_size=32, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)
    
    # Simple test: iterate through one batch.
    for imgs, targets in train_loader:
        print("Train Images shape:", imgs.shape)  # Expected: [batch, 3, 416, 416]
        print("Train Targets:", targets)  # A list, each element a tensor of shape [N, 4] (or [N, 5] if you include classes)
        break


/Users/jamesngugi/miniforge3/envs/ml/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/Users/jamesngugi/miniforge3/envs/ml/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/Users/jamesngugi/miniforge3/envs/ml/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_

Train Images shape: torch.Size([32, 3, 416, 416])
Train Targets: [tensor([], size=(0, 5)), tensor([], size=(0, 5)), tensor([[2.0000, 0.6061, 0.5162, 0.1804, 0.4716]]), tensor([[3.0000, 0.5864, 0.4855, 0.0877, 0.2619]]), tensor([], size=(0, 5)), tensor([], size=(0, 5)), tensor([], size=(0, 5)), tensor([[2.0000, 0.8380, 0.5852, 0.0394, 0.1645],
        [3.0000, 0.6074, 0.7167, 0.1677, 0.3187],
        [3.0000, 0.9053, 0.5950, 0.0623, 0.2260]]), tensor([], size=(0, 5)), tensor([[0.0000, 0.1639, 0.2364, 0.2211, 0.3152],
        [0.0000, 0.3596, 0.3482, 0.3837, 0.3395]]), tensor([], size=(0, 5)), tensor([], size=(0, 5)), tensor([[4.0000, 0.3488, 0.3760, 0.3393, 0.1935]]), tensor([[2.0000, 0.3342, 0.5139, 0.0584, 0.2329]]), tensor([[0.0000, 0.2484, 0.5377, 0.3748, 0.3013],
        [1.0000, 0.3482, 0.4247, 0.1957, 0.6153],
        [1.0000, 0.2662, 0.4531, 0.2402, 0.6257],
        [2.0000, 0.4600, 0.6883, 0.6734, 0.6165],
        [2.0000, 0.2141, 0.3592, 0.2351, 0.4936],
        [4.0000, 0.406

# Training

In [ ]:
import torch
import torch.nn as nn
import timm

class SimpleDetectionHead(nn.Module):
    def __init__(self, backbone_channels, num_classes=6, num_anchors=1): # Simplified: num_anchors=1 for simplicity
        super(SimpleDetectionHead, self).__init__()
        # Reduce channels from backbone
        self.reduce_conv = nn.Conv2d(backbone_channels, 256, kernel_size=1)
        self.relu = nn.ReLU()
        # Bounding box regression - 4 outputs (dx, dy, dw, dh)
        self.bbox_regressor = nn.Conv2d(256, num_anchors * 4, kernel_size=3, padding=1)
        # Objectness confidence - 1 output (probability)
        self.objectness_classifier = nn.Conv2d(256, num_anchors * 1, kernel_size=3, padding=1)
        # Class classification - num_classes outputs
        self.class_classifier = nn.Conv2d(256, num_anchors * num_classes, kernel_size=3, padding=1)
        self.num_classes = num_classes
        self.num_anchors = num_anchors

    def forward(self, features):
        x = self.reduce_conv(features)
        x = self.relu(x)

        bbox_output = self.bbox_regressor(x) # [B, 4, H, W]
        objectness_output = self.objectness_classifier(x) # [B, 1, H, W]
        class_output = self.class_classifier(x) # [B, num_classes, H, W]

        # Reshape for easier handling later, removing anchor dimension if num_anchors=1
        bbox_output = bbox_output.permute(0, 2, 3, 1).contiguous() # [B, H, W, 4]
        objectness_output = objectness_output.permute(0, 2, 3, 1).contiguous() # [B, H, W, 1]
        class_output = class_output.permute(0, 2, 3, 1).contiguous() # [B, H, W, num_classes]

        return bbox_output, objectness_output, class_output


class EfficientNetB0Detector(nn.Module):
    def __init__(self, num_classes=6):
        super(EfficientNetB0Detector, self).__init__()
        # Load EfficientNet-B0 backbone, pretrained, features only, no classifier head
        self.backbone = timm.create_model('efficientnet_b0', pretrained=True, features_only=True, out_indices=(2,)) # Using out_indices=(2,) to get a feature map of reasonable size
        backbone_channels = self.backbone.feature_info.channels()[-1] # Get output channels of the selected feature level (level 2 in this case for efficientnet_b0)
        self.detection_head = SimpleDetectionHead(backbone_channels, num_classes=num_classes)

        # Freeze backbone weights (optional for starter model)
        for param in self.backbone.parameters():
            param.requires_grad = False

    def forward(self, x):
        backbone_features = self.backbone(x)[0] # Get feature map at index 2
        bbox_pred, objectness_pred, class_pred = self.detection_head(backbone_features)
        return bbox_pred, objectness_pred, class_pred # Return as separate outputs


if __name__ == '__main__':
    # Simple test
    model = EfficientNetB0Detector(num_classes=6)
    model.eval() # Set to eval mode for inference

    input_tensor = torch.randn(1, 3, 416, 416) # Batch size 1, 3 channels, 416x416 image
    bbox_output, objectness_output, class_output = model(input_tensor)

    print("Backbone Feature Shape:", model.backbone(input_tensor)[0].shape) # Check backbone feature shape
    print("Bounding Box Output Shape:", bbox_output.shape)      # Expected: [1, H, W, 4]
    print("Objectness Output Shape:", objectness_output.shape)  # Expected: [1, H, W, 1]
    print("Class Output Shape:", class_output.shape)         # Expected: [1, H, W, num_classes]

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


Backbone Feature Shape: torch.Size([1, 40, 52, 52])
Bounding Box Output Shape: torch.Size([1, 52, 52, 4])
Objectness Output Shape: torch.Size([1, 52, 52, 1])
Class Output Shape: torch.Size([1, 52, 52, 6])


In [ ]:
"""
1) Converts VOC XML → YOLO .txt
2) Writes data.yaml
3) Trains YOLOv11 in two phases:
     • Phase 1: freeze backbone, head only
     • Phase 2: unfreeze all, fine-tune
"""

import os
import glob
import xml.etree.ElementTree as ET
import torch

# 1. Install ultralytics if needed:
#    pip install ultralytics

from ultralytics import YOLO

# ─── CONFIG ────────────────────────────────────────────────────────────────────

BASE_DIR        = "/Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data"
IMAGES_DIR      = os.path.join(BASE_DIR, "JPEGImageFull", "dataset", "JPEGImage")
ANNOTATIONS_DIR = os.path.join(BASE_DIR, "positive-Annotation")
LABELS_DIR      = os.path.join(BASE_DIR, "labels")  # where .txt goes
DATA_YAML       = os.path.join(BASE_DIR, "data.yaml")

# Class ↔ ID mapping
CLASS_MAP = {
    "Gun": 0, "Knife": 1, "Wrench": 2,
    "Pliers": 3, "Scissors": 4, "Hammer": 5
}

os.makedirs(LABELS_DIR, exist_ok=True)


# ─── STEP 1: XML → YOLO .TXT (with guards) ────────────────────────────────────

print("Converting XML annotations to YOLO .txt format (skipping malformed objects)...")
for xml_path in glob.glob(os.path.join(ANNOTATIONS_DIR, "*.xml")):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find("size")
    w = float(size.find("width").text)
    h = float(size.find("height").text)

    label_path = os.path.join(
        LABELS_DIR,
        os.path.basename(xml_path).replace(".xml", ".txt")
    )

    with open(label_path, "w") as f:
        for obj in root.findall("object"):
            name_node = obj.find("name")
            if name_node is None or name_node.text not in CLASS_MAP:
                # skip objects without a valid class
                continue
            cls_name = name_node.text
            cid = CLASS_MAP[cls_name]

            bnd = obj.find("bndbox")
            if bnd is None:
                # skip objects without a bounding-box
                continue

            try:
                xmin = float(bnd.find("xmin").text)
                ymin = float(bnd.find("ymin").text)
                xmax = float(bnd.find("xmax").text)
                ymax = float(bnd.find("ymax").text)
            except (AttributeError, TypeError):
                # if any coordinate is missing, skip this object
                continue

            # Normalize to YOLO format
            x_center = ((xmin + xmax) / 2) / w
            y_center = ((ymin + ymax) / 2) / h
            bw = (xmax - xmin) / w
            bh = (ymax - ymin) / h

            f.write(f"{cid} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")


import pandas as pd

# ─── STEP 2: CSV SPLITS + GUARANTEE .TXT + WRITE data.yaml ────────────────────
print("Generating splits, creating missing .txt labels, and writing data.yaml…")

# 1) Paths to your CSVs
TRAIN_CSV = os.path.join(BASE_DIR, "TestTrainSplits", "train_test_easy", "train-3000.csv")
VAL_CSV   = os.path.join(BASE_DIR, "TestTrainSplits", "train_test_easy", "test-3000.csv")

# 2) Read the image IDs
train_ids = pd.read_csv(TRAIN_CSV)['name'].tolist()
val_ids   = pd.read_csv(VAL_CSV)['name'].tolist()

# 3) Write train.txt & val.txt (absolute paths to .jpg)
train_list = os.path.join(BASE_DIR, "train.txt")
val_list   = os.path.join(BASE_DIR, "val.txt")

with open(train_list, 'w') as f:
    for img_id in train_ids:
        f.write(os.path.join(IMAGES_DIR, f"{img_id}.jpg") + "\n")

with open(val_list, 'w') as f:
    for img_id in val_ids:
        f.write(os.path.join(IMAGES_DIR, f"{img_id}.jpg") + "\n")

# 4) Ensure every split image has *some* .txt label (empty if no objects)
for img_id in set(train_ids + val_ids):
    lbl_path = os.path.join(LABELS_DIR, f"{img_id}.txt")
    if not os.path.isfile(lbl_path):
        open(lbl_path, 'w').close()

# 5) Write data.yaml with an explicit path:
print("Writing data.yaml…")
# ─── WRITE data.yaml ─────────────────────────────────────────────────────────
data_yaml = f"""\
train: {train_list}
val:   {val_list}
nc:    {len(CLASS_MAP)}
names: {list(CLASS_MAP.keys())}
"""
with open(DATA_YAML, "w") as f:
    f.write(data_yaml)
print("Wrote data.yaml without a `path:`. YOLOv11 will now use your split files directly.")


print("STEP 2 complete.")


# ─── STEP 3: INITIALIZE & TRAIN YOLOv11 ────────────────────────────────────────

# Choose your variant: nano / small / medium / large / xlarge
PRETRAINED = "yolo11l.pt"

print(f"Loading YOLOv11 model ({PRETRAINED})...")
model = YOLO(PRETRAINED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on device: {device}")

# Phase 1: freeze backbone → train head only
print("Phase 1: freezing backbone, training head...")
model.train(
    data=DATA_YAML,
    epochs=2,
    imgsz=416,
    batch=16,
    lr0=1e-3,
    optimizer="AdamW",
    freeze=10,          # freeze first 10 layers of backbone
    device=device,
    project="runs/yolo11",
    name="phase1",
    exist_ok=True
)

# Phase 2: unfreeze all → fine-tune end-to-end
print("Phase 2: unfreezing all layers, fine-tuning entire model...")
model.train(
    data=DATA_YAML,
    epochs=4,
    imgsz=416,
    batch=8,
    lr0=1e-4,
    optimizer="AdamW",
    device=device,
    project="runs/yolo11",
    name="phase2",
    exist_ok=True
)

print("Training complete. Models and logs are in runs/yolo11/phase{1,2}/")


Converting XML annotations to YOLO .txt format (skipping malformed objects)...
Generating splits, creating missing .txt labels, and writing data.yaml…
Writing data.yaml…
Wrote data.yaml without a `path:`. YOLOv11 will now use your split files directly.
STEP 2 complete.
Loading YOLOv11 model (yolo11l.pt)...
Training on device: cpu
Phase 1: freezing backbone, training head...
New https://pypi.org/project/ultralytics/8.3.119 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.105 🚀 Python-3.10.16 torch-2.4.1.post2 CPU (Apple M4)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data/data.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=runs/yolo11, name=phase1, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False,

train: Scanning /Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data/JPEGImageFull/dataset/JPEGImage... 14122 images, 7078 backgrounds, 0 corrupt: 100%|██████████| 14122/14122 [00:01<00:00, 7293.93it/s]


train: New cache created: /Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data/JPEGImageFull/dataset/JPEGImage.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data/JPEGImageFull/dataset/JPEGImage... 3532 images, 1770 backgrounds, 0 corrupt: 100%|██████████| 3532/3532 [00:00<00:00, 5739.92it/s]

val: New cache created: /Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data/JPEGImageFull/dataset/JPEGImage.cache
Plotting labels to runs/yolo11/phase1/labels.jpg... 


...

        4/4         0G      1.479      1.321      1.553          4        416: 100%|██████████| 1766/1766 [1:35:17<00:00,  3.24s/it]    
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 221/221 [30:15<00:00,  8.21s/it] 

                   all       3532       3513      0.759      0.649      0.737      0.455



4 epochs completed in 11.862 hours.
Optimizer stripped from runs/yolo11/phase2/weights/last.pt, 51.2MB
Optimizer stripped from runs/yolo11/phase2/weights/best.pt, 51.2MB

Validating runs/yolo11/phase2/weights/best.pt...
Ultralytics 8.3.105 🚀 Python-3.10.16 torch-2.4.1.post2 CPU (Apple M4)
YOLO11l summary (fused): 190 layers, 25,283,938 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 221/221 [22:51<00:00,  6.21s/it] 


                   all       3532       3513      0.759       0.65      0.737      0.455
                   Gun        593        932      0.871      0.942      0.969      0.651
                 Knife        407        622      0.862       0.63      0.781      0.473
                Wrench        425        614      0.655      0.539      0.621      0.401
                Pliers        803       1109      0.743      0.609      0.716      0.421
              Scissors        204        236      0.666       0.53        0.6      0.328
Speed: 0.2ms preprocess, 294.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/yolo11/phase2
Training complete. Models and logs are in runs/yolo11/phase{1,2}/


# Testing

In [ ]:
# from ultralytics import YOLO

# load your best Phase 2 weights
model = YOLO("runs/yolo11/phase2/weights/best.pt")

# run validation on your splits
# this re-runs the same evaluation you saw at train-time
metrics = model.val(
    data="/Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data/data.yaml",
    imgsz=416,
    batch=16
)

print(metrics)  # dict with P, R, mAP50, mAP50-95 per class + overall


Ultralytics 8.3.105 🚀 Python-3.10.16 torch-2.4.1.post2 CPU (Apple M4)
YOLO11l summary (fused): 190 layers, 25,283,938 parameters, 0 gradients


val: Scanning /Users/jamesngugi/Desktop/Applied ML/ML-Project/test-data/JPEGImageFull/dataset/JPEGImage.cache... 3532 images, 1770 backgrounds, 0 corrupt: 100%|██████████| 3532/3532 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 221/221 [17:12<00:00,  4.67s/it]


                   all       3532       3513      0.759       0.65      0.737      0.455
                   Gun        593        932      0.871      0.942      0.969      0.651
                 Knife        407        622      0.862       0.63      0.781      0.473
                Wrench        425        614      0.655      0.539      0.621      0.401
                Pliers        803       1109      0.743      0.609      0.716      0.421
              Scissors        204        236      0.666       0.53        0.6      0.328
Speed: 0.1ms preprocess, 289.9ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x35f7a10f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: 